This notebook calculates the following:
### <p><center>For each genre, what is the gender split among all artists that are labeled with that genre?</center></p> 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
#%matplotlib notebook


import re

from functools import partial

import plotly.graph_objects as go

Import the cleaned data:

In [2]:
#%ls -lt ../../data/genre_lists/data_ready_for_model/

In [3]:
%store -r now
now
#now = '2020-05-11-14-35'

'2020-05-18-10-06'

In [4]:
X_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_X_train_{}.csv'.format(now), index_col = ['artist'])
y_train = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/wiki-kaggle_y_train_{}.csv'.format(now), index_col = ['artist'])

### Genre Labels -- as a list

Each value of the genre column is a _string_ of comma separated genre labels using the spotify abbreviations. We want to convert it to a _list_ of strings.

In [5]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label."""
def genrelist(string):
    string = string.strip("[").strip("]").replace("'","")
    L = [s for s in string.split(',')]
    L_new = []
    for x in L:
        L_new.append(x.replace(" ","_").lstrip("_").rstrip("_"))
    while (str("") in L_new):
        L_new.remove("")
    return L_new

Now we apply
- it to the whole column and put the lists in a new column
- assemble X,y into DF
-reset index to 'artist_id'

In [6]:
X_train['genrelist']= X_train['genrelist'].apply(genrelist)

data = X_train.join(y_train, how = 'inner', on = 'artist')

data.reset_index(inplace = True)
data.index.name = 'artist_id'
data_set_size = data.shape[0]

- Full genre_list (not just that for the training set)
- Vocab Dict and Size
- max length of lists

In [7]:
genre_list = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_list_{}.csv'.format(now))
genre_list.drop(['Unnamed: 0'], axis = 1, inplace = True)
genre_list['genre_id'] = list(range(1,genre_list.shape[0]+1))

#Size of the vocab:
vocab_size = genre_list.shape[0]

#Create a dictionary {genre_label: genre_id}
label_id_dict = genre_list.set_index(['genre_list'])['genre_id'].to_dict()
id_label_dict = genre_list.set_index(['genre_id'])['genre_list'].to_dict()



#Find max length of genre lists:
max_list_length = data.genrelist_length.max()

### Count thefrequency of each label and prepare other columns

(This deals only with the training data, not the test data.)

In [8]:
genre_list_1 = data.genrelist.values.tolist()
genre_list_1 = [x for y in genre_list_1 for x in y]
genre_counts = pd.Series(genre_list_1, name = 'frequency')
genre_stats = genre_counts.value_counts().to_frame()
genre_stats.index.name = 'label'

In [9]:
# encode labels as ints within the list
def encode_list(row):
    return [label_id_dict[item] for item in row.genrelist]

data['genres_encoded_as_list'] = data.apply(encode_list, axis = 1)

#Check that the encoding is consistent: 
# n = np.random.randint(data.shape[0])
# [label_id_dict[item] for item in data.genrelist.iloc[n]], data.genres_encoded_as_list.iloc[n]

# Encode targets. The categories still appear as strings. To see the encoding use df.column.cat.codes.
data['gender'] = data.gender.apply(lambda x: 1 if x == 'female' else 0)

In [10]:
max_num_male = 73
max_num_female = 11
max_num = max(max_num_female, max_num_male)

### Stats by that genre label
- [ ] do this using sparse dataframe with one-hot encoding for speed up?

In [11]:
# fnc used with apply to data to select artists with a given label
def indicate(row):
    if label in row.genrelist:
        return 1
    else:
        return 0

In [12]:
data.columns

Index(['artist', 'genrelist', 'genrelist_length', 'gender',
       'genres_encoded_as_list'],
      dtype='object')

In [13]:
# produce stats for each label: male, female; mean number of labels; max, min number of labels
idx_list = []
for label in genre_stats.index: # use labels ordered by their frequency of appearance
    data['indicator'] = data.apply(indicate, axis = 1)
    label_artists = data[data.indicator == 1]
    genre_stats.loc[label,'female'] = int(label_artists.gender.sum())
    genre_stats.loc[label,'male'] = label_artists.shape[0]-label_artists.gender.sum()
    genre_stats.loc[label,'max_list_length'] = label_artists.genrelist_length.max()
    genre_stats.loc[label,'min_list_length'] = label_artists.genrelist_length.min()
    genre_stats.loc[label,'mean_list_length'] = label_artists.genrelist_length.mean()
    #data.drop(['indicator'], inplace = True)

# calculated columns
genre_stats['female%'] = genre_stats['female']/genre_stats['frequency']
genre_stats['male%'] = genre_stats['male']/genre_stats['frequency']
# reorder columns
genre_stats = genre_stats[['frequency','female','male','female%','male%','max_list_length','min_list_length','mean_list_length']]

Write to csv:

In [15]:
genre_stats.to_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_stats.csv')

In [16]:
genre_stats = pd.read_csv('/Users/Daniel/Code/Genre/data/genre_lists/data_ready_for_model/genre_stats.csv', index_col = 'label')

In [17]:
genre_stats.head()

,frequency,female,male,female%,male%,max_list_length,min_list_length,mean_list_length
label,,,,,,,,
pop,2617,1321.0,1294.0,0.504776,0.494459,73.0,1.0,3.065392
rock,1765,356.0,1409.0,0.201700,0.798300,13.0,1.0,3.373371
r_and_b,1647,760.0,887.0,0.461445,0.538555,13.0,1.0,3.585914
country,1613,504.0,1108.0,0.312461,0.686919,12.0,1.0,2.358561
hip_hop,1114,187.0,927.0,0.167864,0.832136,73.0,1.0,2.538600


Overall ratio to total of female and male artists:

In [22]:
f_ratio = data.gender.sum()/data.shape[0]
m_ratio = (1-data.gender).sum()/data.shape[0]
f_ratio, m_ratio

(0.3108435681965094, 0.6891564318034906)

In [23]:
genre_stats['f-skew'] = genre_stats['female%']/f_ratio
genre_stats['m-skew'] = genre_stats['male%']/m_ratio

In [24]:
genre_stats.head()

,frequency,female,male,female%,male%,max_list_length,min_list_length,mean_list_length,f-skew,m-skew
label,,,,,,,,,,
pop,2617,1321.0,1294.0,0.504776,0.494459,73.0,1.0,3.065392,1.623892,0.717485
rock,1765,356.0,1409.0,0.201700,0.798300,13.0,1.0,3.373371,0.648879,1.158373
r_and_b,1647,760.0,887.0,0.461445,0.538555,13.0,1.0,3.585914,1.484493,0.781470
country,1613,504.0,1108.0,0.312461,0.686919,12.0,1.0,2.358561,1.005204,0.996753
hip_hop,1114,187.0,927.0,0.167864,0.832136,73.0,1.0,2.538600,0.540026,1.207471
